In [ ]:
import json
import numpy as np
import pandas as pd
import unidecode
from pandas.io.json import json_normalize


## Creating the news dataframe
The first step is import the news dataset and creating a Pandas Dataframe

In [ ]:
news_data_file = open('../datasets/raw-data/folha_de_sao_paulo_news.json', 'r')
news_json = json.load(news_data_file)

news_df = json_normalize(news_json['news'])

news_df.head(5)

# Selecting only the columns that we are going to feed into the Neural Network

In [ ]:
news_df['text'] = news_df['title'] + '. ' + news_df['sub_title'] '. ' + news_df['news_body']
news_df = news_df[['date_published', 'text']]

news_df.head(5)

# Saving the clean dataset into a file

In [ ]:
news_df.to_csv('../datasets/pre-processed/news.csv', index= False, encoding='utf-8')